In [1]:
import os
import json
import torch
from tqdm import tqdm
from model_utils import *
from data_utils import *
from prompt_utils import *
torch.set_grad_enabled(False)
from baukit import TraceDict, get_module
import torch.nn as nn

In [2]:
model_name = 'meta-llama/Llama-3.1-8B'
dataset_name = 'banking77_3token'
data_dir = './dataset'
num_shots_by_class = 2
demon_selection='stratify'
seed=42

set_seed(seed)
train_dataset, valid_dataset, test_dataset, label_list = load_data(dataset_name, data_dir)
device = 'cuda'
model, tokenizer, model_config = load_model_and_tokenizer(model_name)
for param in model.parameters():
    param.requires_grad = False
model.to(device)
model.set_attn_implementation("eager") 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
for test_item in tqdm(test_dataset):
    test_prompt, test_query_target = create_prompt(demon_pool=train_dataset, query = test_item, num_shots_by_class=num_shots_by_class, option=demon_selection, label_list=label_list, shuffle_label=False)
    test_tokenized_input = tokenizer(test_prompt, return_tensors='pt').to(device)
    with torch.no_grad():
        o = model.generate(**test_tokenized_input, return_dict_in_generate = True, output_attentions=True, max_new_tokens=3, pad_token_id=tokenizer.eos_token_id, eos_token_id = tokenizer.eos_token_id, tokenizer=tokenizer, do_sample=False, temperature = None, top_p = None)   
    outputs = o.sequences
    attentions = o.attentions    
    output_str = tokenizer.decode(outputs.squeeze()[test_tokenized_input.input_ids[0].shape[0]:], skip_speical_tokens=True)
    print(test_item)
    print(test_tokenized_input.input_ids[0].shape[0])
    for step in range(len(attentions)):
        cur_attn_scores = attentions[step][14][0,8,-1,:]
        values, indices = torch.topk(cur_attn_scores, k=10)
        print(f"step {step}")
        print(values.tolist())
        print(indices.tolist())
        print("___________")

  0%|          | 1/1039 [00:00<16:54,  1.02it/s]

{'input': 'How do I locate my card?', 'output': 'card_arrival'}
996
step 0
[0.349609375, 0.28515625, 0.171875, 0.09228515625, 0.020751953125, 0.01953125, 0.0093994140625, 0.005615234375, 0.0042724609375, 0.0036468505859375]
[993, 0, 995, 992, 991, 990, 986, 994, 989, 984]
___________
step 1
[0.3828125, 0.314453125, 0.09130859375, 0.080078125, 0.0625, 0.011474609375, 0.00823974609375, 0.005615234375, 0.00482177734375, 0.00274658203125]
[995, 0, 996, 993, 992, 994, 990, 991, 986, 984]
___________
step 2
[0.34375, 0.28515625, 0.1318359375, 0.0947265625, 0.052001953125, 0.021728515625, 0.01007080078125, 0.00701904296875, 0.005218505859375, 0.004791259765625]
[996, 995, 0, 993, 997, 992, 986, 994, 984, 965]
___________


  0%|          | 2/1039 [00:01<10:41,  1.62it/s]

{'input': 'I still have not received my new card, I ordered over a week ago.', 'output': 'card_arrival'}
980
step 0
[0.322265625, 0.2119140625, 0.1796875, 0.17578125, 0.0166015625, 0.0162353515625, 0.01611328125, 0.005584716796875, 0.004852294921875, 0.004486083984375]
[0, 979, 977, 978, 976, 969, 970, 960, 958, 975]
___________
step 1
[0.447265625, 0.232421875, 0.1259765625, 0.076171875, 0.048583984375, 0.01171875, 0.0089111328125, 0.00531005859375, 0.0029754638671875, 0.0026702880859375]
[979, 0, 980, 978, 977, 969, 976, 970, 958, 960]
___________
step 2
[0.400390625, 0.189453125, 0.18359375, 0.09228515625, 0.057861328125, 0.033203125, 0.0059814453125, 0.003692626953125, 0.0032501220703125, 0.00194549560546875]
[980, 0, 979, 981, 978, 977, 969, 976, 970, 960]
___________


  0%|          | 3/1039 [00:01<08:46,  1.97it/s]

...

{'input': 'still waiting on that card', 'output': 'card_arrival'}
1056
step 0
[0.298828125, 0.22265625, 0.1982421875, 0.1171875, 0.0673828125, 0.0302734375, 0.01287841796875, 0.009033203125, 0.0079345703125, 0.006683349609375]
[0, 1053, 1055, 1054, 1052, 1049, 1051, 1050, 1048, 1047]
___________
step 1
[0.41796875, 0.26171875, 0.09765625, 0.055419921875, 0.054931640625, 0.0498046875, 0.01226806640625, 0.00592041015625, 0.00579833984375, 0.004180908203125]
[1055, 0, 1056, 1052, 1053, 1054, 1049, 1050, 1051, 1047]
___________
step 2
[0.314453125, 0.2373046875, 0.07080078125, 0.029052734375, 0.023681640625, 0.020263671875, 0.0186767578125, 0.017333984375, 0.0166015625, 0.012939453125]
[0, 1056, 1055, 1057, 1047, 1045, 1053, 1052, 1046, 1029]
___________


  1%|          | 10/1039 [00:04<06:03,  2.83it/s]

{'input': 'Is it normal to have to wait over a week for my new card?', 'output': 'card_arrival'}
1001
step 0
[0.322265625, 0.3046875, 0.25390625, 0.0274658203125, 0.017333984375, 0.00628662109375, 0.005645751953125, 0.00555419921875, 0.004669189453125, 0.004119873046875]
[998, 0, 1000, 997, 993, 995, 999, 996, 990, 982]
___________
step 1
[0.515625, 0.2265625, 0.10888671875, 0.06787109375, 0.0220947265625, 0.0091552734375, 0.0048828125, 0.003814697265625, 0.003692626953125, 0.00286865234375]
[1000, 0, 1001, 998, 997, 993, 999, 995, 990, 994]
___________
step 2
[0.41796875, 0.232421875, 0.11572265625, 0.023193359375, 0.01275634765625, 0.01263427734375, 0.01141357421875, 0.01129150390625, 0.0078125, 0.007659912109375]
[1001, 0, 1000, 998, 997, 999, 1002, 982, 947, 981]
___________


  1%|          | 10/1039 [00:04<07:34,  2.26it/s]


KeyboardInterrupt: 

In [108]:
test_tokenized_input.input_ids[0].shape[0]

996

In [107]:
for l in range(model_config['n_layers']):
    for h in range(model_config['n_heads']):
        print(l,h)
        for step in range(len(attentions)):
            cur_attn_scores = attentions[step][l][0,h,-1,:]
            values, indices = torch.topk(cur_attn_scores, k=10)
            print(f"step {step}")
            print(values.tolist())
            print(indices.tolist())
            print("___________")


0 0
step 0
[0.8125, 0.0245361328125, 0.0223388671875, 0.0223388671875, 0.016845703125, 0.016357421875, 0.015869140625, 0.0135498046875, 0.0123291015625, 0.0108642578125]
[993, 991, 992, 990, 994, 995, 984, 986, 987, 989]
___________
step 1
[0.400390625, 0.142578125, 0.1337890625, 0.059326171875, 0.0478515625, 0.03955078125, 0.028076171875, 0.0186767578125, 0.01409912109375, 0.01324462890625]
[993, 995, 994, 992, 991, 990, 984, 989, 985, 986]
___________
step 2
[0.26953125, 0.173828125, 0.126953125, 0.08447265625, 0.0439453125, 0.038818359375, 0.03759765625, 0.0322265625, 0.0322265625, 0.01336669921875]
[993, 995, 994, 996, 997, 992, 990, 991, 984, 985]
___________
0 1
step 0
[0.171875, 0.15234375, 0.1220703125, 0.111328125, 0.10107421875, 0.0634765625, 0.04638671875, 0.0218505859375, 0.0164794921875, 0.01416015625]
[993, 994, 984, 995, 978, 970, 964, 0, 952, 986]
___________
step 1
[0.11865234375, 0.111328125, 0.111328125, 0.1015625, 0.05615234375, 0.051025390625, 0.0478515625, 0.03002

In [4]:
from typing import List, Dict, Tuple, Any, Union
from dataclasses import dataclass
from transformers import AutoTokenizer

@dataclass
class Span:
    demo_index: Union[int, str]   # 1-based index, or "test"
    component: str                # one of the 6 components below
    start: int                    # char start (inclusive)
    end: int                      # char end (exclusive)

COMPONENTS = [
    "input_template", "input_sentence", "input_separator",
    "output_template", "output_sentence", "output_separator"
]

def parse_examples_exact(
    text: str,
    input_label: str = "Input:",
    output_label: str = "Output:",
    input_sep: str = "\n",
    output_sep: str = "\n\n",
    strict_singleline: bool = True,   # 엄격히 "한 줄 + 지정 개행"을 따름
) -> List[Span]:
    """
    네가 정의한 단위:
      Input template   : "Input:"
      Input sentence   : label 뒤부터 첫 '\n' 직전까지 (앞 공백 포함 가능)
      Input separator  : 정확히 '\n'
      Output template  : "Output:"
      Output sentence  : label 뒤부터 첫 '\n\n' 직전까지 (앞 공백 포함 가능)
      Output separator : 정확히 '\n\n'

    strict_singleline=True  -> 반드시 위 규칙을 강제.
    strict_singleline=False -> 예외적으로 input sentence가 여러 줄일 경우, Output: 직전까지를 sentence로 간주.
    """
    spans: List[Span] = []
    pos = 0
    demo_idx = 1
    N = len(text)

    while True:
        i_start = text.find(input_label, pos)
        if i_start == -1:
            break
        i_end = i_start + len(input_label)

        # Input sentence: until first '\n'
        nl_pos = text.find(input_sep, i_end)
        if nl_pos == -1:
            # Input만 있고 개행이 없는 비정상: template만 기록하고 종료
            spans.append(Span(demo_index=demo_idx, component="input_template", start=i_start, end=i_end))
            break

        # 기본(엄격) 규칙: 한 줄 sentence + '\n'는 separator
        i_sent_start = i_end
        i_sent_end = nl_pos
        i_sep_start = nl_pos
        i_sep_end = nl_pos + len(input_sep)

        # Output template는 input separator 직후에 있어야 함(중간 공백/빈 줄 없이)
        # 단, 완화 모드에서는 Output:가 더 뒤에 있어도 허용
        o_start = text.find(output_label, i_sep_end)
        if o_start == -1:
            # Output 라벨이 없으면 input쪽만 기록
            spans.append(Span(demo_index=demo_idx, component="input_template", start=i_start, end=i_end))
            spans.append(Span(demo_index=demo_idx, component="input_sentence", start=i_sent_start, end=i_sent_end))
            spans.append(Span(demo_index=demo_idx, component="input_separator", start=i_sep_start, end=i_sep_end))
            break

        # 완화 옵션: 만약 strict이고 i_sep_end != o_start라면 포맷 위반으로 간주
        #           strict_singleline=False면, Input sentence를 Output 직전까지 확장하고,
        #           separator는 빈 문자열(없음)로 처리하지 않고, 첫 '\n'만 separator로 남긴다(정의 고수).
        if strict_singleline:
            # strict 모드: Output 라벨은 바로 다음 줄 시작이어야 함
            # (i_sep_end == o_start)인 경우가 가장 깔끔하지만, 중간 공백이 있어도 허용할지 선택 가능.
            pass
        else:
            if i_sep_end < o_start:
                # sentence를 Output 라벨 직전까지 확장 (개행 포함)
                i_sent_end = o_start
                # separator는 여전히 첫 '\n'만 기록(정의 유지)
                # (추가 개행이 sentence에 포함될 수 있음)

        o_end = o_start + len(output_label)

        # Output sentence: until first '\n\n'
        out_sep_pos = text.find(output_sep, o_end)
        next_i = text.find(input_label, o_end)

        # test query 판정: Output 뒤 텍스트가 없고 다음 Input도 없음(문서 끝)
        # 혹은 Output 뒤에 공백뿐이고 다음 Input 없음
        # => demo_index="test", output_sentence 비어있음, output_separator 없음(정의상)
        tail = text[o_end:N]
        is_test = (next_i == -1) and ( (out_sep_pos == -1) and (tail.strip() == "") )

        # 기록: Input 3요소
        cur_demo = "test" if is_test else demo_idx
        spans.append(Span(demo_index=cur_demo, component="input_template", start=i_start, end=i_end))
        spans.append(Span(demo_index=cur_demo, component="input_sentence", start=i_sent_start, end=i_sent_end))
        spans.append(Span(demo_index=cur_demo, component="input_separator", start=i_sep_start, end=i_sep_end))

        # 기록: Output template
        spans.append(Span(demo_index=cur_demo, component="output_template", start=o_start, end=o_end))

        if is_test:
            # test query: output sentence/sep 없음
            break

        # 일반 케이스: output sentence와 separator
        if out_sep_pos != -1 and (next_i == -1 or out_sep_pos < next_i):
            # 정상: output sentence는 o_end ~ out_sep_pos, separator는 그 '\n\n'
            spans.append(Span(demo_index=demo_idx, component="output_sentence", start=o_end, end=out_sep_pos))
            spans.append(Span(demo_index=demo_idx, component="output_separator", start=out_sep_pos, end=out_sep_pos + len(output_sep)))
            pos = out_sep_pos + len(output_sep)
        else:
            # '\n\n'이 없거나 다음 Input이 먼저 오면: sentence를 Output~다음 Input(or 끝)까지로
            out_end = next_i if next_i != -1 else N
            # sentence 기록
            spans.append(Span(demo_index=demo_idx, component="output_sentence", start=o_end, end=out_end))
            # separator는 정의상 '\n\n'이 없으므로 생략
            pos = out_end

        demo_idx += 1

    return spans

def build_token_entity_map(
    prompt_text: str,
    tokenizer,
    add_special_tokens: bool = False,
    **parse_kwargs
) -> Tuple[Dict[int, Dict[str, Any]], List[Span], Dict[str, Any]]:
    """
    반환:
      token_map: { token_idx: {"demo_index": int|"test", "component": COMPONENTS[str]} }
      spans    : 문자 단위 스팬(디버깅용)
      token_info: {"input_ids": [...], "offsets": [(s,e), ...]}
    """
    spans = parse_examples_exact(prompt_text, **parse_kwargs)

    enc = tokenizer(prompt_text, return_offsets_mapping=True, add_special_tokens=add_special_tokens)
    offsets = enc["offset_mapping"]
    input_ids = enc["input_ids"]

    # 토큰 인덱스 → 스팬 매핑
    def overlaps(tok_se, sp: Span) -> bool:
        s, e = tok_se
        return (s < sp.end) and (sp.start < e)

    token_map: Dict[int, Dict[str, Any]] = {}
    for ti, se in enumerate(offsets):
        s, e = se
        if s == e:  # 특수토큰 등
            continue
        for sp in spans:
            if overlaps((s, e), sp):
                token_map[ti] = {"demo_index": sp.demo_index, "component": sp.component}
                break

    token_info = {"input_ids": input_ids, "offsets": offsets}
    return token_map, spans, token_info

In [5]:
token_map, spans, token_info = build_token_entity_map(
    test_prompt, tokenizer,
    # 아래는 포맷 정의(네가 지정한 기본값과 동일)
    input_label="Input:",
    output_label="Output:",
    input_sep="\n",
    output_sep="\n\n",
    strict_singleline=True,  # 필요시 False로 완화
)

# 앞 40개만 프린트
for i in range(len(token_info["input_ids"])):
    tid = token_info["input_ids"][i]
    piece = tokenizer.convert_ids_to_tokens([tid])[0]
    meta = token_map.get(i)
    print(f"{i:>4}  {piece:<12}  {meta}")

   0  Input         {'demo_index': 1, 'component': 'input_template'}
   1  :             {'demo_index': 1, 'component': 'input_template'}
   2  ĠHow          {'demo_index': 1, 'component': 'input_sentence'}
   3  Ġdo           {'demo_index': 1, 'component': 'input_sentence'}
   4  ĠI            {'demo_index': 1, 'component': 'input_sentence'}
   5  Ġtop          {'demo_index': 1, 'component': 'input_sentence'}
   6  Ġup           {'demo_index': 1, 'component': 'input_sentence'}
   7  ?Ċ            {'demo_index': 1, 'component': 'input_sentence'}
   8  Output        {'demo_index': 1, 'component': 'output_template'}
   9  :             {'demo_index': 1, 'component': 'output_template'}
  10  Ġtransfer     {'demo_index': 1, 'component': 'output_sentence'}
  11  _into         {'demo_index': 1, 'component': 'output_sentence'}
  12  _account      {'demo_index': 1, 'component': 'output_sentence'}
  13  ĊĊ            {'demo_index': 1, 'component': 'output_separator'}
  14  Input         {'demo_

In [85]:
for step in range(len(attentions)):
    attn = attentions[step]
    head_attention_dict = {}
    for l in range(model_config['n_layers']):
        for h in range(model_config['n_heads']):
            cur_attn_scores = attn[l][0,h,-1,:]
            values, indices = torch.topk(cur_attn_scores, k=10)
            top_attended_token_dict=[]
            for v,i in zip(values, indices):
                value = v.item()
                idx = i.item()
                if idx==0:
                    piece = tokenizer.bos_token
                    meta = "BOS_token"
                else:
                    tid = token_info["input_ids"][idx-1]
                    piece = tokenizer.convert_ids_to_tokens([tid])[0]
                    meta = token_map.get(idx-1)
                top_attended_token_dict.append({"token": piece, "score":value, "meta":meta})
            top_attended_token_dict = sorted(top_attended_token_dict, key=lambda x: x['score'], reverse=True)
            head_attention_dict[f"({l},{h})"] = top_attended_token_dict
    break

In [86]:
test_prompt

"Input: How do I top up?\nOutput: transfer_into_account\n\nInput: I keep being asked to verify my identity. Is this normal?\nOutput: why_verify_identity\n\nInput: How can I exchange USD and GBP in the app?\nOutput: exchange_via_app\n\nInput: What is the limit on Auto top?\nOutput: automatic_top_up\n\nInput: why hasn't my virtual card arrived?\nOutput: getting_virtual_card\n\nInput: I tried to top-up but it got denied.\nOutput: top_up_failed\n\nInput: Is it possible to track the card you sent me?\nOutput: card_arrival\n\nInput: Should I keep waiting for my card to be delivered?\nOutput: card_arrival\n\nInput: Why hasnt my top up gone through yet?? I thought it was suppose to be instant? Am I wrong? Why is it still pending?\nOutput: pending_top_up\n\nInput: I already have a card and want to link it\nOutput: card_linking\n\nInput: Changed details is essential, what steps do I take to do so?\nOutput: edit_personal_details\n\nInput: HOW LONG THE CARD VALIDITY\nOutput: card_not_working\n\nIn

In [106]:
d = head_attention_dict["(26,3)"]
for item in d:
    print(item)
    print("___")

{'token': '<|begin_of_text|>', 'score': 0.41015625, 'meta': 'BOS_token'}
___
{'token': ':', 'score': 0.37890625, 'meta': {'demo_index': 'test', 'component': 'output_template'}}
___
{'token': '?Ċ', 'score': 0.03173828125, 'meta': {'demo_index': 'test', 'component': 'input_sentence'}}
___
{'token': 'Output', 'score': 0.009765625, 'meta': {'demo_index': 'test', 'component': 'output_template'}}
___
{'token': 'Ġmy', 'score': 0.00872802734375, 'meta': {'demo_index': 'test', 'component': 'input_sentence'}}
___
{'token': '_card', 'score': 0.00836181640625, 'meta': {'demo_index': 47, 'component': 'output_sentence'}}
___
{'token': 'Input', 'score': 0.006866455078125, 'meta': {'demo_index': 14, 'component': 'input_template'}}
___
{'token': 'Output', 'score': 0.00653076171875, 'meta': {'demo_index': 35, 'component': 'output_template'}}
___
{'token': 'Output', 'score': 0.004974365234375, 'meta': {'demo_index': 40, 'component': 'output_template'}}
___
{'token': '?Ċ', 'score': 0.00494384765625, 'meta

In [66]:
alpha = torch.load("./result/banking77_3token/llama3.1-8b/alpha.pt")
alpha = torch.sigmoid(alpha)
group_1 = [[],[],[]]
group_2 = [[],[],[]]
group_3 = [[],[],[]]
for t in range(3):
    alphas = alpha[t]
    for l in range(28):
        alpha_by_layer = alphas[l]
        for h in range(28):
            alpha_by_head = alpha_by_layer[h]
            if 0.1<alpha_by_head.item()<0.9:
                group_3[t].append((l,h))
            elif alpha_by_head.item()>=0.9:
                group_1[t].append((l,h))
            else:
                group_2[t].append((l,h))

In [101]:
alpha[0][14,2]

tensor(0.6970, device='cuda:0')

In [67]:
group_3[0]

[(4, 15),
 (5, 5),
 (6, 3),
 (7, 23),
 (8, 7),
 (9, 2),
 (9, 10),
 (10, 21),
 (11, 22),
 (12, 13),
 (12, 20),
 (14, 2),
 (14, 8),
 (14, 13),
 (15, 7),
 (15, 17),
 (16, 20),
 (20, 14),
 (22, 0),
 (22, 1),
 (23, 5),
 (24, 16),
 (25, 7),
 (26, 3),
 (27, 4)]